In [2]:
import config

In [3]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

# A non programmer can also create a chatbot
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

In [4]:
def collect_messages():
    prompt = ""
    while True:
        prompt = input("User Input: ")
        context.append({'role': 'user', 'content': f"{prompt}"})
        response = get_completion_from_messages(context)
        context.append({'role': 'assistant', 'content': f"{response}"})
        display(response)
        if ("Goodbye" in response) or ("Have a great day" in response) or ("Thank You" in response) or ("We accept online payments" in response) or ("Thank you for choosing our pizza restaurant" in response) or prompt in ("bye", "exit"):
            break
    return response

response = collect_messages()

'Hello! Welcome to our pizza restaurant. How can I assist you today?'

'Great choice! I have noted down your order for a large cheese pizza with no additional toppings. It will be ready for pickup at 3 pm today. Is there anything else you would like to add to your order?'

'Alright, I have your order for a large cheese pizza with no additional toppings for pickup at 3 pm today. Is there anything else I can assist you with?'

'Alright, I have your order for a large cheese pizza with no additional toppings for pickup at 3 pm today. If you have any other questions or need further assistance, feel free to let me know. Thank you for choosing our pizza restaurant!'

In [5]:
messages =  context.copy()
messages.append(
{
    'role':'system', 
    'content':'create a json summary of the previous food order. Itemize the price for each item\
    The fields should be \
    1) pizza, include size \
    2) list of toppings \
    3) list of drinks, include size  \
    4) list of sides include size  \
    5) total price\
    6) name\
    7) phone\
    8) address \
    9) mode of payment'},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "size": "large",
    "type": "cheese"
  },
  "toppings": [],
  "drinks": [],
  "sides": [],
  "total_price": 10.95,
  "name": "John Doe",
  "phone": "123-456-7890",
  "address": "123 Main St",
  "payment_method": "cash"
}


In [83]:
# Convert the JSON object to a dictionary
data = json.loads(response)
order=pd.json_normalize(data)
df=pd.read_csv('orders.csv')
df.iloc[0,:].index=['toppings', 'drinks', 'sides', 'total_price', 'name', 'phone',
       'address', 'payment_method', 'pizza.size', 'pizza.type']
order.to_csv('orders.csv',mode='a',header=False,index=False)

In [1]:
df=pd.read_csv('orders.csv')
df

NameError: name 'pd' is not defined